In [ ]:
import yaml

%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")
import pcse
from pcse.models import Wofost72_PP
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse.fileinput import YAMLCropDataProvider
from pcse.util import WOFOST72SiteDataProvider, DummySoilDataProvider
from progressbar import printProgressBar

print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations

In [ ]:
# Define location, crop and season
latitude = 32.108136
longitude = -7.842794
crop_name = 'wheat'
variety_name = 'Winter_wheat_101'
campaign_start_date = '2022-01-01'
emergence_date = "2022-03-31"
harvest_date = "2022-10-20"
max_duration = 300

In [ ]:
# Here we define the agromanagement for sugar beet
agro_yaml = """
- {start}:
    CropCalendar:
        crop_name: {cname}
        variety_name: {vname}
        crop_start_date: {startdate}
        crop_start_type: emergence
        crop_end_date: {enddate}
        crop_end_type: harvest
        max_duration: {maxdur}
    TimedEvents: null
    StateEvents: null
""".format(cname=crop_name, vname=variety_name, 
           start=campaign_start_date, startdate=emergence_date, 
           enddate=harvest_date, maxdur=max_duration)
agro = yaml.safe_load(agro_yaml)
print(agro_yaml)

In [ ]:
# Weather data for Netherlands
wdp = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)

# Parameter sets for crop, soil and site
# Standard crop parameter library
cropd = YAMLCropDataProvider()

# We don't need soil for potential production, so we use dummy values
soild = DummySoilDataProvider()
# Some site parameters
sited = WOFOST72SiteDataProvider(WAV=50)

# Retrieve all parameters in the form of a single object. 
# In order to see all parameters for the selected crop already, we
# synchronise data provider cropd with the crop/variety: 
firstkey = list(agro[0])[0]
cropcalendar = agro[0][firstkey]['CropCalendar'] 
cropd.set_active_crop(cropcalendar['crop_name'], cropcalendar['variety_name'])
params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

In [ ]:
wofost = Wofost72_PP(params, wdp, agro)
wofost.run_till_terminate()

df_results = pd.DataFrame(wofost.get_summary_output())
#df_results = df_results.set_index("day")
df_results.tail()

In [ ]:
df_results_out = pd.DataFrame(wofost.get_output())
df_results_out = df_results_out.set_index("day")
df_results_out.tail()

In [ ]:
from pcse.fileinput import CABOFileReader

cropfile = os.path.join(data_dir, 'crop', 'SUG0601.crop')
cropd = CABOFileReader(cropfile)

In [ ]:
soilfile = os.path.join(data_dir, 'soil', 'ec3.soil')
soild = CABOFileReader(soilfile)

In [ ]:
from pcse.util import WOFOST72SiteDataProvider
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)

In [ ]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropd, soildata=soild, sitedata=sited)

In [ ]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

In [ ]:
from pcse.fileinput import CSVWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'wdata.csv')
wdp = CSVWeatherDataProvider(weatherfile)
print(wdp)

In [ ]:
from pcse.fileinput import ExcelWeatherDataProvider
weatherfile = os.path.join(data_dir, 'meteo', 'wdata.xlsx')
wdp = ExcelWeatherDataProvider(weatherfile)
print(wdp)

In [ ]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
wofsim = Wofost72_WLP_FD(parameters, data, agromanagement)

In [ ]:
wofsim.run_till_terminate()
df_results = pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

In [ ]:
from get_market_value import *

agrom = {
    'latitude' : 37.5453,
    'longitude' : -7.6765,
    'crop_area' : 12,
    'crop_name' : "wheat",
    'variety_name' : "Winter_wheat_103",
    'campaign_start_date' : "2022-01-01",
    'emergence_date' : "2022-03-31",
    'harvest_date' : "2022-12-20",
    'max_duration' : 300}


print(get_market_value(agrom), "$")

In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

countyBoundary = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq('STATEFP', '55')) \
    .filter(ee.Filter.eq('NAME', 'Dane'))

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(countyBoundary, 10)

Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')

# Display the map
Map.addLayerControl()
Map

In [ ]:
from get_land_area import *

get_land_area(countyBoundary)

In [ ]:
import datetime as dt

start_date = dt.date(2022, 8, 20)

print(start_date)

In [7]:
def get_temp_ind(crop_geometry, start_date, end_date):

    # Define the base and upper temperature thresholds (in Celsius)
    base_temp = 5.0
    upper_temp = 30.0

    # Define the lower temperature threshold (in Celsius)
    lower_temp = 0.0

    # Filter MODIS Land Surface Temperature (MOD11A2) imagery by date and AOI
    filtered_collection = ee.ImageCollection('MODIS/006/MOD11A1') \
        .filterDate(start_date, end_date) \
        .filterBounds(crop_geometry)

    # Function to calculate GDD, EHDD, and ECDD for an image
    def calculate_indices(image):
        # Retrieve the temperature band
        lst = image.select('LST_Day_1km')

        print(lst)

        # Calculate GDD, EHDD, and ECDD
        gdd = ee.Image(0).where(lst.gt(base_temp), lst.subtract(base_temp).divide(2)).rename('GDD')
        ehdd = ee.Image(0).where(lst.gt(upper_temp), lst.subtract(upper_temp)).rename('EHDD')
        ecdd = ee.Image(0).where(lst.lt(lower_temp), lower_temp.subtract(lst)).rename('ECDD')

        # Return the image with added GDD, EHDD, and ECDD bands
        return image.addBands([gdd, ehdd, ecdd])

    # Map the calculate_indices function over the image collection
    indices_collection = filtered_collection.map(calculate_indices)

    # Get the GDD, EHDD, and ECDD bands from the first image in the collection
    first_image = ee.Image(indices_collection.first())
    gdd_band = first_image.select('GDD')
    ehdd_band = first_image.select('EHDD')
    ecdd_band = first_image.select('ECDD')

    # Calculate the cumulative sum of GDD, EHDD, and ECDD over time
    cumulative_gdd = indices_collection.select('GDD').sum().rename('Cumulative_GDD')
    cumulative_ehdd = indices_collection.select('EHDD').sum().rename('Cumulative_EHDD')
    cumulative_ecdd = indices_collection.select('ECDD').sum().rename('Cumulative_ECDD')

    # Print the cumulative GDD, EHDD, and ECDD images
    print('Cumulative GDD:', cumulative_gdd.getInfo())
    print('Cumulative EHDD:', cumulative_ehdd.getInfo())
    print('Cumulative ECDD:', cumulative_ecdd.getInfo())


In [2]:
import geemap

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')

Map.addLayerControl()
# Display the map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [18]:
import ee

crop_geometry = ee.FeatureCollection(Map.draw_features)

plant_date = '2021-09-01'
har_date = '2022-07-20'

In [50]:
cumulative_ecdd_values